In [1]:
pip install torch torchvision onnx onnx-tf tensorflow pillow opencv-python



INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


In [1]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models

import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

# -------------------------------
# CONFIGURATIONS
# -------------------------------
DATASET_PATH = r"C:/Users/georg/OneDrive/Desktop/Capsule/Dataset 2"
MODEL_DIR = "trained_models"
PYTORCH_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.pth")
ONNX_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.onnx")
TF_SAVED_MODEL_PATH = os.path.join(MODEL_DIR, "tf_saved_model")
TFLITE_MODEL_PATH = os.path.join(MODEL_DIR, "mobilenetv2_skin_tone.tflite")

BATCH_SIZE = 32
NUM_CLASSES = 3
EPOCHS = 80
LEARNING_RATE = 0.001
RANDOM_SEED = 42
VAL_SPLIT = 0.2

os.makedirs(MODEL_DIR, exist_ok=True)

# -------------------------------
# SET DEVICE (Forces CUDA if available)
# -------------------------------
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Training on:", device)

# -------------------------------
# DATA AUGMENTATION AND LOAD DATASET
# -------------------------------
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

full_dataset = datasets.ImageFolder(DATASET_PATH, transform=data_transforms)
class_to_idx = full_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Split into train/val sets
val_size = int(len(full_dataset) * VAL_SPLIT)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=torch.cuda.is_available())
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=torch.cuda.is_available())

print(f"Total images: {len(full_dataset)} | Training: {len(train_dataset)} | Validation: {len(val_dataset)}")

# -------------------------------
# MODEL SETUP
# -------------------------------
from torchvision.models import MobileNet_V2_Weights
model = models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# -------------------------------
# TRAINING AND VALIDATION LOOP
# -------------------------------
for epoch in range(EPOCHS):
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (preds == labels).sum().item()
    scheduler.step()
    train_acc = 100 * train_correct / train_total
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()
    val_acc = 100 * val_correct / val_total
    print(f"Epoch [{epoch+1}/{EPOCHS}]: Train Loss {train_loss/train_total:.4f} | Train Acc {train_acc:.2f}% | "
          f"Val Loss {val_loss/val_total:.4f} | Val Acc {val_acc:.2f}% | LR {scheduler.get_last_lr()[0]:.5f}")

# -------------------------------
# SAVE PYTORCH MODEL
# -------------------------------
torch.save(model.state_dict(), PYTORCH_MODEL_PATH)
print(f"Trained PyTorch model saved to: {PYTORCH_MODEL_PATH}")

# -------------------------------
# EXPORT TO ONNX
# -------------------------------
model.eval()
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(
    model,
    dummy_input,
    ONNX_MODEL_PATH,
    input_names=['input'],
    output_names=['output'],
    opset_version=11,
    do_constant_folding=True,
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)
print(f"Model exported to ONNX format at: {ONNX_MODEL_PATH}")

# -------------------------------
# CONVERT ONNX TO TensorFlow SavedModel
# -------------------------------
onnx_model = onnx.load(ONNX_MODEL_PATH)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(TF_SAVED_MODEL_PATH)
print(f"TensorFlow SavedModel exported at: {TF_SAVED_MODEL_PATH}")

# -------------------------------
# CONVERT TensorFlow SavedModel TO TFLITE
# -------------------------------
converter = tf.lite.TFLiteConverter.from_saved_model(TF_SAVED_MODEL_PATH)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open(TFLITE_MODEL_PATH, "wb") as f:
    f.write(tflite_model)
print(f"TensorFlow Lite model saved to: {TFLITE_MODEL_PATH}")


C:\Users\georg\anaconda3\envs\tf15\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\georg\anaconda3\envs\tf15\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you 

Training on: cuda
Total images: 852 | Training: 682 | Validation: 170
Epoch [1/80]: Train Loss 0.4780 | Train Acc 81.96% | Val Loss 0.2142 | Val Acc 92.94% | LR 0.00100
Epoch [2/80]: Train Loss 0.1279 | Train Acc 95.31% | Val Loss 0.1311 | Val Acc 96.47% | LR 0.00100
Epoch [3/80]: Train Loss 0.0717 | Train Acc 97.80% | Val Loss 0.0297 | Val Acc 99.41% | LR 0.00100
Epoch [4/80]: Train Loss 0.0931 | Train Acc 96.33% | Val Loss 0.0598 | Val Acc 97.65% | LR 0.00100
Epoch [5/80]: Train Loss 0.0509 | Train Acc 98.24% | Val Loss 0.1208 | Val Acc 96.47% | LR 0.00100
Epoch [6/80]: Train Loss 0.0433 | Train Acc 98.53% | Val Loss 0.0469 | Val Acc 97.65% | LR 0.00100
Epoch [7/80]: Train Loss 0.0398 | Train Acc 98.83% | Val Loss 0.1185 | Val Acc 95.29% | LR 0.00100
Epoch [8/80]: Train Loss 0.0827 | Train Acc 97.07% | Val Loss 0.0941 | Val Acc 96.47% | LR 0.00100
Epoch [9/80]: Train Loss 0.0785 | Train Acc 97.65% | Val Loss 0.2448 | Val Acc 94.12% | LR 0.00100
Epoch [10/80]: Train Loss 0.0550 | Trai

INFO:tensorflow:Assets written to: trained_models\tf_saved_model\assets


TensorFlow SavedModel exported at: trained_models\tf_saved_model
TensorFlow Lite model saved to: trained_models\mobilenetv2_skin_tone.tflite
